**Grupo 11**

Paula Alejandra Cadena Espitia

César Felipe Pineda Ortiz

Maria Fernanda Carbonell Santos

In [2]:
import signal


class TimeoutError(Exception):
    def __init__(self, value="Timed Out"):
        self.value = value

    def __str__(self):
        return repr(self.value)


def timeout(seconds_before_timeout):
    def decorate(f):
        def handler(signum, frame):
            raise TimeoutError()

        def new_f(*args, **kwargs):
            old = signal.signal(signal.SIGALRM, handler)
            signal.alarm(seconds_before_timeout)
            try:
                result = f(*args, **kwargs)
            finally:
                signal.signal(signal.SIGALRM, old)
            signal.alarm(0)
            return result

        new_f.func_name = f.func_name
        return new_f

    return decorate


In [3]:
import itertools



timeout(5 * 60)
def bruteforce(x_list, target):
    possiblities = []
    for x in powerset(x_list):
        possiblities.append((x, sum(x)))

    x_list, actual_value = closest(possiblities, target)

    return (actual_value, x_list)


def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(itertools.combinations(s, r) for r in range(len(s) + 1))


def closest(possiblities, target):
    return min((abs(target - total), (o_list, total))
               for o_list, total in possiblities)[1]

In [4]:
def stackoverflow(x_list, target):
    memo = dict()
    result, _ = g(x_list, x_list, target, memo)
    return (sum(result), result)


def g(v, w, S, memo):
    subset = []
    id_subset = []
    for i, (x, y) in enumerate(zip(v, w)):
        # Check if there is still a solution if we include v[i]
        if f(v, i + 1, S - x, memo) > 0:
            subset.append(x)
            id_subset.append(y)
            S -= x
    return subset, id_subset


def f(v, i, S, memo):
    if i >= len(v):
        return 1 if S == 0 else 0
    if (i, S) not in memo:    # <-- Check if value has not been calculated.
        count = f(v, i + 1, S, memo)
        count += f(v, i + 1, S - v[i], memo)
        memo[(i, S)] = count  # <-- Memoize calculated result.
    return memo[(i, S)]       # <-- Return memoized value.


In [5]:
import operator


def approx_with_accounting_and_duplicates(x_list,
                                s,      # target value
                                ):
    '''
    Modified from http://en.wikipedia.org/wiki/Subset_sum_problem#Polynomial_time_approximate_algorithm

         initialize a list S to contain one element 0.
         for each i from 1 to N do
           let T be a list consisting of xi + y, for all y in S
           let U be the union of T and S
           sort U
           make S empty
           let y be the smallest element of U
           add y to S
           for each element z of U in increasing order do
              //trim the list by eliminating numbers close to one another
              //and throw out elements greater than s
             if y + cs/N < z <= s, set y = z and add z to S
         if S contains a number between (1 - c)s and s, output yes, otherwise no

    '''
    c = .01              # fraction error (constant)
    N = len(x_list)      # number of values

    S = [(0, [])]
    for x in sorted(x_list):
        T = []
        for y, y_list in S:
            T.append((x + y, y_list + [x]))
        U = T + S
        U = sort_by_col(U, 0)
        y, y_list = U[0]
        S = [(y, y_list)]

        for z, z_list in U:
            lower_bound = (float(y) + c * float(s) / float(N))
            if lower_bound < z <= s:
                y = z
                S.append((z, z_list))

    return sort_by_col(S, 0)[-1]


def sort_by_col(table, col=0):
    '''
    http://www.saltycrane.com/blog/2007/12/how-to-sort-table-by-columns-in-python/
    '''
    return sorted(table, key=operator.itemgetter(col))


In [6]:
from __future__ import division
from heapq import heappush, heappop
from itertools import count


def bb_knapsack(v, c):
    sol = [0]                                     # Solution so far (global var)
    n = len(v)                                    # Item count

    idxs = list(range(n))

    def bound(sv, m):                             # Greedy knapsack bound
        if m == n:
            return sv                             # No more items?
        objs = ((v[i], 'dum') for i in idxs[m:])  # Descending unit cost order
        for av, _ in objs:                        # Added value and weight
            if sv + av > c:
                break                             # Still room?
            sv += av                              # Add val to sum of vals
        return sv + (c - sv)                      # Add fraction of last item

    def node(sv, m):                              # A node (generates children)
        if sv > c:
            return                                # Weight sum too large? Done
        sol[0] = max(sol[0], sv)                  # Otherwise: Update solution
        if m == n:
            return                                # No more objects? Return
        i = idxs[m]                               # Get the right index
        ch = [('dum', sv), ('dum', sv + v[i])]    # Children: without/with m
        for _, sv in ch:                          # Try both possibilities
            b = bound(sv, m + 1)                  # Bound for m+1 items
            if b > sol[0]:                        # Is the branch promising?
                yield b, node(sv, m + 1)          # Yield child w/bound

    num = count()                                 # Helps avoid heap collisions
    Q = [(0, next(num), node(0, 0))]              # Start with just the root
    while Q:                                      # Any nodes left?
        _, _, r = heappop(Q)                      # Get one
        for b, u in r:                            # Expand it ...
            heappush(Q, (b, next(num), u))        # ... and push the children

    return sol[0]                                 # Return the solution


In [7]:
from time import time


def main():
    # x_list = [100, 75, 15, 495, 995, 995, 995, 995, 510, 110]
    # target = 635
    x_list = [1150, 495, 995, 995, 995, 995, 100, 750, 3305, 75, 510, 3265, 2145, 1935, 140, 140, 15, 1330, 2800, 1250, 350, 850, 110]
    target = 30

    time0 = time()
    bf = bruteforce(x_list, target)
    time1 = time()
    so = stackoverflow(x_list, target)
    time2 = time()
    he = bb_knapsack(x_list, target)
    time3 = time()
    wi = approx_with_accounting_and_duplicates(x_list, target)
    time4 = time()

    print ('Brute force:', bf, time1 - time0)
    print ('Stackoverflow:', so, time2 - time1)
    print ('Hetland:', he, time3 - time2)
    print ('Wikipedia:', wi, time4 - time3)


if __name__ == '__main__':
    main()

Brute force: (15, (15,)) 10.605933427810669
Stackoverflow: (0, []) 0.034430503845214844
Hetland: 15 0.00015974044799804688
Wikipedia: (15, [15]) 0.00012755393981933594
